In [195]:
# %pip install pywin32
# %pip install openpyxl
# %pip install schedule

In [196]:
import win32com.client

outlook = win32com.client.Dispatch("Outlook.Application")
namespace = outlook.GetNamespace("MAPI")
inbox = namespace.Folders("RU: Service Desk, RUSSIA").Folders("Входящие")
#namespace.GetDefaultFolder(6)

In [197]:
# Получаем сообщения из папки "Входящие"
messages = inbox.Items
messages.sort("[ReceivedTime]", True)

# Проверяем, есть ли непрочитанные письма
# unread_count = sum(1 for message in messages if message.Unread)
# print(f"Количество непрочитанных сообщений: {unread_count}")

message = messages[0]
print(f"Тема: {message.Subject}, От: {message.SenderName}, Дата: {message.ReceivedTime}")


Тема: смена учетной записи, От: Khismatullina,Elina,RU-Moscow, Дата: 2024-12-24 13:15:37.593000+00:00


In [198]:
import pandas as pd
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("./sentence_bert_service_desk_model_finetuned")
df = pd.read_excel("TEMPLATES.xlsx")

answers = df["Текст"].tolist()
link_names = df["Название ссылки"].tolist()
links = df["Ссылка"].tolist()

answer_embeddings = model.encode(answers)

In [199]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def generate_reply(input_text, answer_embeddings, threshold=0.3):
    question_embedding = model.encode(input_text)
    similarity_matrix = cosine_similarity(
        question_embedding.reshape(1, -1), answer_embeddings
    )[0]
    # print(similarity_matrix)

    # Вычисляем медиану и межквартильный размах (IQR)
    median_similarity = np.median(similarity_matrix)
    q1 = np.percentile(similarity_matrix, 25)
    q3 = np.percentile(similarity_matrix, 75)
    iqr = q3 - q1

    # Устанавливаем порог как медиана + 1.5 * IQR
    threshold = max(median_similarity + 1.25 * iqr, threshold)
    valid_answers = [
        (sim, ans, link, link_name)
        for sim, ans, link, link_name in zip(
            similarity_matrix, answers, links, link_names
        )
        if sim > threshold
    ]
    if len(valid_answers) == 1:
        response = "<p>Добрый день!</p>"
        response += f"<p>{valid_answers[0][1]}<br><a href='{valid_answers[0][2]}'>{valid_answers[0][3]}</a></p>"
        response += "<p><i>Искренне ваш,<br/><u>ServiceBot</u>, самый бездушный сотрудник Service Desk.</i></p>"
        return response
    elif len(valid_answers) > 1:
        valid_answers.sort(reverse=True, key=lambda x: x[0])
        top_answers = valid_answers[:3]

        response = "<p>Добрый день!</p>"
        response += "<p>Наш робот не смог найти точный ответ на ваш запрос. Скорее всего, вам подойдёт один из ответов ниже:</p>"
        response += "<ul>"

        for _, answer, link, link_name in top_answers:
            response += f"<li>{answer}<br/><a href='{link}'>{link_name}</a></li>"

        response += "</ul>"
        response += "<p><i>Искренне ваш,<br/><u>ServiceBot</u>, самый бездушный сотрудник Service Desk.</i></p>"
        return response.strip()
    else:
        # print("No answer found")
        return "-1"

In [200]:
banned_subjects = [
    "RE:",
    "FWD:",
    "INC",
    "REQ",
    "Recall:",
    "Undeliverable:",
    "Notification",
]
banned_senders = [
    "RuPull@nestlesoft.net",
    "Microsoft Outlook",
    "ELMA365",
    "Nestle Service-Now",
]

In [201]:
import json
from datetime import datetime

# Для теста будем запоминать пользователей + темы писем
testing_bag = set() # Убрать на проде
def check_inbox():
    outlook = win32com.client.Dispatch("Outlook.Application")
    namespace = outlook.GetNamespace("MAPI")
    inbox = namespace.Folders("RU: Service Desk, RUSSIA").Folders("Входящие")
    messages = inbox.Items
    messages.sort("[ReceivedTime]", True)
    # Читаем последние 100 сообщений
    for i in range(100):
        message = messages[i]
        # Если message -- действительно сообщение, а не всякое там
        if message.Class == 43:
            is_not_banned_sender = not any(
                banned_sender in message.SenderName for banned_sender in banned_senders
            )
            is_not_banned_subject = not any(
                banned_subject in message.Subject for banned_subject in banned_subjects
            )
            if (
                message.Unread
                and is_not_banned_sender
                and is_not_banned_subject
                and (message.SenderName, message.Subject) not in testing_bag # Убрать на проде
            ):
                process_message(message)
                testing_bag.add((message.SenderName, message.Subject)) # Убрать на проде


def process_message(message):
    msg = message.Subject + " " + message.Body
    reply = generate_reply(msg, answer_embeddings, 0.3)
    # Логирование
    if True:
        log_entry = {"query": msg, "answers": [{"text": reply, "label": ""}]}
        log_file_path = "message_log.txt"
        with open(log_file_path, "a", encoding="utf-8") as log_file:
            log_file.write(json.dumps(log_entry, ensure_ascii=False) + "\n")
    if reply != "-1":
        send_reply(message, reply)


def send_reply(original_message, reply_body):
    reply = original_message.Reply()
    reply.To = "RUPostamAt1@ru.nestle.com"
    reply.Subject = "Re: " + original_message.Subject
    original_head = f"<p><b>From:</b> {original_message.SenderName} <br><b>Sent:</b> {original_message.ReceivedTime}<br><b>To:</b> {message.To}<br><b>Subject:</b> {original_message.Subject}</p>"
    original_body = original_message.HTMLBody
    reply.HTMLBody = f"<p><font size='3'>{reply_body}</font></p><br/><hr/>{original_head}{original_body}"
    reply.Send()
    
    # Оставляем непрочитанным - True, помечаем как прочитанное - False
    original_message.Unread = True
    original_message.Save()

In [202]:
check_inbox()

In [203]:
import schedule
import time

# Должен быть открыт Outlook
# check_inbox()
def job():
    current_time = datetime.now().time()
    if (
        current_time >= datetime.strptime("07:00", "%H:%M").time()
        and current_time <= datetime.strptime("20:00", "%H:%M").time()
    ):
        check_inbox()
        print(datetime.now().time())


# Запуск функции job каждые 5 минут
schedule.every(5).minutes.do(job)

while True:
    schedule.run_pending()
    time.sleep(1)

13:17:35.418585
13:17:36.179819


KeyboardInterrupt: 